In [12]:
import pandas as pd

In [17]:
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import os

mysql_config = {
    "host": "localhost",
    "port": 13308,
    "database": "tgdd",
    "user": "root",
    "password": "admin"  
}

names = ['brands', 'products', 'products_spec', 'products_rating']
path = './data/processed/'
try:
    engine = create_engine(f"mysql+mysqlconnector://{mysql_config['user']}:{mysql_config['password']}@{mysql_config['host']}:{mysql_config['port']}/{mysql_config['database']}")
    with engine.connect() as conn:

            for name in names:
                csv_file_name =  f"{name}.json"
                table_name = name
                csv_file_path = os.path.join(path, csv_file_name)
                df = pd.read_json(csv_file_path, orient='records', lines=True)
                df.to_sql(
                    table_name, 
                    con=conn, 
                    if_exists='append', 
                    index=False
                )
            print("Data loaded sucessfully!")
    
except Error as e:
    print("Error occured:",e)
    
try:
    with  mysql.connector.connect(**mysql_config) as connection:
    
        try:
            cursor = connection.cursor()
            cursor.execute("""ALTER TABLE brands
                              ADD PRIMARY KEY (ID);

                              ALTER TABLE products
                              ADD PRIMARY KEY (ID);

                              ALTER TABLE products
                              ADD CONSTRAINT fk_brands_products
                              FOREIGN KEY (BrandID) REFERENCES brands(ID);

                              ALTER TABLE products_spec
                              ADD PRIMARY KEY (ProductID, Name);

                              ALTER TABLE products_spec
                              ADD CONSTRAINT fk_products_products_spec
                              FOREIGN KEY (ProductID) REFERENCES products(ID);

                              ALTER TABLE products_rating
                              ADD PRIMARY KEY (ProductID, Name);

                              ALTER TABLE products_rating
                              ADD CONSTRAINT fk_products_products_rating
                              FOREIGN KEY (ProductID) REFERENCES products(ID);    
                        """)
        
        except Exception as e:
            print("Error occured:",e)
            
        finally:
            cursor.close()
except Error as e:
    print("Error occured:",e)
                

Data loaded sucessfully!
